In [27]:
from gensim.models.word2vec import Word2Vec
from gensim.models.phrases import Phrases, Phraser
import os, json, tqdm, re, itertools, nltk

In [22]:
def read_file(file_path):
    all_words_from_files = []
    with open (file_path) as file:
        json_content = json.load(file)
        item_count = 0
        for item in json_content:
            if item_count < 10:
                item_count += 1
                text_content = re.sub("<.*?>", "", item["textContent"])
                text_content = text_content.replace('-\n', '')
                text_content = text_content.replace("\n", '')
                
#                 print (text_content)
#             text_content = [word for word in text_content if len(word) > 0]
            all_words_from_files.append(text_content)
    return all_words_from_files
       
        
def read_all_judgments_from_2018(path):
    all_judgments = []
    for filename in tqdm.tqdm(os.listdir(path)):
        judgments = read_file(path + filename)
        all_judgments.append(judgments)
    return all_judgments

# We need a stream of sentences
def flatten_judgments(words):
    chain = list(itertools.chain(*words))
    result = []
    for judgment in chain:
        for word in judgment:
            result.append(word)
    return result
        

In [23]:
words = read_all_judgments_from_2018("../data_filtered_2/")

100%|██████████| 11/11 [00:00<00:00, 31.62it/s]


In [28]:
from nltk.tokenize import TweetTokenizer, sent_tokenize

tokenizer_words = TweetTokenizer()
tokens_sentences = [tokenizer_words.tokenize(t) for t in 
nltk.sent_tokenize(words)]
print(tokens_sentences)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  Searched in:
    - '/home/grzegorz/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/home/grzegorz/anaconda2/envs/py3/nltk_data'
    - '/home/grzegorz/anaconda2/envs/py3/lib/nltk_data'
    - ''
**********************************************************************


In [114]:
words = flatten_judgments(words)
words[:10]

[' ', ' ', ' ', ' ', ' ', ' ', 'U', 'Z', 'A', 'S']

In [108]:
phrases = Phrases(words, threshold=10000)
bigram = Phraser(phrases)

In [111]:
print (bigram["ala", "ma", "new", "york"])

['ala', 'ma', 'new', 'york']
